This notebook is the next step of the methodology. We have, from the previous step, xyz files of lat/lon/z values (mean thicknesses, carbon content, age etc.). Here we take these xyz files and using GMT turn them into rasters. We apply a block median to organise the points a bit better, then use spherical interpolation to make the grid. We output them to a separate folder. These rasters then let us calcualte the amount of carbon/serpentinite/[z value] that intersects with subduction zones, a certain age of lithosphere at a given time (i.e. how much, and where is the serpentinite that is 50 Ma old at 70 Ma).

I hungrily await the day for pyGMT.

In [ ]:
import numpy as np
import pandas as pd
import pygplates
import os

In [ ]:
datadir = '/Users/Andrew/Documents/PhD/Scripts/Python_Scripts/pyGPlates_examples/Merdith_2019_Frontiers/Sample_Data/'
datadir_results = '/Users/Andrew/Documents/PhD/Scripts/Python_Scripts/pyGPlates_examples/Merdith_2019_Frontiers/output/x_y_z_model/'
savedir = '/Users/Andrew/Documents/PhD/Scripts/Python_Scripts/pyGPlates_examples/Merdith_2019_Frontiers/output/rasters/'

rotfile = '%sMatthews_etal_GPC_2016_410-0Ma_GK07.rot' % datadir
coastlines = '%sGlobal_EarthByte_GeeK07_COB_Terranes_Matthews_etal.gpml' % datadir

rotation_model = pygplates.RotationModel(rotfile)
#rotation_model_palaeozoic = pygplates.RotationModel(rotfile_palaeozoic)
coastlines = pygplates.FeatureCollection(coastlines)
polygons = []
for feature in coastlines:
    for geom in feature.get_geometries():
        polygon = feature
        polygon.set_geometry(pygplates.PolygonOnSphere(geom))
        polygons.append(polygon)

coastlines = pygplates.FeatureCollection(polygons)

In [ ]:
def gmt_processing(time, parameters, savedir, COB_infile):

    '''
    uses GMT to make rasters at set times based on gridded interpolation of xyz points created
    in previous notebook
    '''
    #loaddir_results = savedir
    masked_COBs = '%smask_COB_%s.nc' % (datadir_results, time) #masking grids
    infiles = [] #the results of the analysis we pass in
    lst_of_bm_outfiles = [] #our blockmedian tmp files, deleted asap
    lst_of_grids = [] #our regular gridded files using sphinterpolate
    #outfiles_COB = [] #our final processed files after applying two masks (nn and COB)
    outfiles_NN_COB = [] #our final processed files after applying two masks (nn and COB)
    
    #set some gmt parameters
    region='-180./180./-90./90.'
    resolution = '0.5'
    
    #we have a list of the parameters that are traced in each grid, so we populate each empty list with
    #the appropriate file
    for i in parameters:
        infiles.append('%sPMSR_%s_%s_Ma' % (datadir_results, i,time))
        lst_of_bm_outfiles.append('%sPMSR_%s_%s_Ma-bm_tmp' % (datadir_results, i,time))
        lst_of_grids.append('%sPMSR_%s_%s_Ma.grd' % (savedir, i,time))
        outfiles_NN_COB.append('%sPMSR_%s_%s_Ma_NN_COB_masked.grd' % (savedir, i,time))
        #outfiles_COB.append('%sPMSR_%s_%s_Ma_COB_masked.grd' % (savedir, i,time))

    #print outfiles    
    #need to delete existing files for gmt to have an attempt
    for ind, i in enumerate(lst_of_bm_outfiles):
        if os.path.isfile(i[ind]):
            os.remove(i)
        if os.path.isfile(lst_of_bm_outfiles[ind]):
            os.remove(lst_of_bm_outfiles[ind])
        if os.path.isfile(outfiles_NN_COB[ind]):
            os.remove(outfiles_NN_COB[ind])  
        #if os.path.isfile(outfiles_COB[ind]):
        #    os.remove(outfiles_COB[ind])  

    #COB mask
    os.system('gmt grdmask %s -R%s -I%s -N1/0/0 -V -G%s' % (COB_infile, region, resolution, masked_COBs))

    for infile,bm_tmp in zip(infiles,lst_of_bm_outfiles):
        os.system('gmt blockmedian %s -I%s -R%s -V > %s' % (infile, resolution, region, bm_tmp))
    for bm_tmp,grid_outfile in zip(lst_of_bm_outfiles, lst_of_grids):
        #print bm_tmp, grid_outfile
        os.system('gmt sphinterpolate %s -G%s -I%s -R%s -V' % (bm_tmp, grid_outfile, resolution, region))
  
    #near neighbour mask
    #they should all be the same, so we only need one filter
    nn_outfile = '%sPMSR_Serp_%s_Ma_NN.grd' % (savedir,time) #make the grid
    os.system('gmt nearneighbor %s -N4 -I%s -R%s -E0 -S3d -G%s' % (lst_of_bm_outfiles[0], resolution, region, nn_outfile))
    nn_mask = '%sPMSR_Serp_%s_Ma_NN_mask.grd' % (savedir,time) #make the mask
    os.system('gmt grdmath %s %s DIV = %s' % (nn_outfile, nn_outfile, nn_mask))
    for grid,NN_COB_outfile in zip(lst_of_grids, outfiles_NN_COB):
        os.system('gmt grdmath %s %s MUL %s MUL = %s' % (nn_mask, masked_COBs, grid, NN_COB_outfile))
    #for grid,COB_outfile in zip(lst_of_grids, outfiles_COB):
    #    os.system('gmt grdmath %s %s MUL = %s' % (masked_COBs, grid, COB_outfile))

        #remove chunky files
    for bm_tmp in lst_of_bm_outfiles:#, lst_of_grids):
        #if os.path.isfile('%s' % grid):
        #    os.remove('%s' % grid)
        if os.path.isfile('%s' % bm_tmp):
            os.remove('%s' % bm_tmp)

    return


In [ ]:
times = np.arange(0,201,1)
#parameters = ['C', 'Serp', 'upper', 'SR']
parameters = ['Serp', 'total', 'upper', 'lower', 'dykes', 'gabbro', 'SR', 'age']

for time in times:
    print time
    pygplates.reconstruct(coastlines,
                      rotation_model,
                      '%sreconstructed_COB_%s.gmt' % (savedir, time),
                      time)
    COB_gmt = '%sreconstructed_COB_%s.gmt' % (savedir, time)
        
    gmt_processing(time, parameters, savedir, COB_gmt)